In [17]:
import pandas as pd
import numpy as np
import re
import Levenshtein
from datetime import datetime
from thefuzz import process
import os
import yaml

In [18]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%autoreload 2

In [20]:
from utils import *
import utils
print(dir(utils))

['Levenshtein', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'categorisation_logic', 'import_transactions', 'load_categories', 'os', 'pd', 'preprocess_desc', 'primary_match', 'process', 'secondary_match', 'sequence_keywords', 'spaceless_strings', 'yaml']


In [21]:
transactions_df = import_transactions()
# transactions_df.info()

In [22]:
# clean descriptions
transactions_df['string_keywords'] = preprocess_desc(transactions_df)
# display(transactions_df['string_keywords'].head())

In [23]:
# load categories
categories = load_categories()
# display(categories.tail())

In [24]:
# write the best match for each keyword
best_matches = primary_match(categories['brand'], transactions_df['string_keywords'])
finance_data_score_df = pd.merge(transactions_df, best_matches, how='inner', right_index=True, left_on='string_keywords')
# display(finance_data_score_df.head())

In [25]:
dataset_keywords = sequence_keywords(transactions_df['string_keywords'])
# display(dataset_keywords.head())

In [26]:
best_word_match_df = secondary_match(categories['brand'], dataset_keywords)

finance_matches_df = pd.merge(
    finance_data_score_df, 
    best_word_match_df, 
    how='inner', 
    left_index=True, 
    right_on='description_index'
)
finance_matches_df.drop('description_index', axis=1, inplace=True)
finance_matches_df.loc[finance_matches_df['string_match_brand'] == finance_matches_df['word_match_brand'], 'methods_match'] = True

In [27]:
finance_matches_df['brand_final'] = finance_matches_df.apply(categorisation_logic, axis=1)

In [30]:
# display(finance_matches_df.head())

In [29]:
file_path = os.path.join('..', 'datasets', 'finance_matches_df_output.csv')
finance_matches_df.to_csv(file_path)